In [1]:
import numpy as np
import torchvision
import cv2 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import os
import torch

In [2]:
!unzip /content/lesson1_dataset.zip

Archive:  /content/lesson1_dataset.zip
   creating: logloss_1/
  inflating: logloss_1/m3.png        
  inflating: logloss_1/logloss_1logloss_1_34.png  
   creating: __MACOSX/
   creating: __MACOSX/logloss_1/
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_34.png  
  inflating: logloss_1/logloss_1logloss_1_20.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_20.png  
  inflating: logloss_1/logloss_1logloss_1_21.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_21.png  
  inflating: logloss_1/logloss_1logloss_1_35.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_35.png  
  inflating: logloss_1/m2.png        
  inflating: logloss_1/logloss_1logloss_1_23.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_23.png  
  inflating: logloss_1/logloss_1logloss_1_37.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_37.png  
  inflating: logloss_1/logloss_1logloss_1_36.png  
  inflating: __MACOSX/logloss_1/._logloss_1logloss_1_36.png  
  inflating: l

In [3]:
def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y

In [4]:
class MyClass(Dataset):
    def __init__(self, X, Y):
        self.x = torch.from_numpy(X)
        self.y = torch.from_numpy(Y)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return len(self.x)

In [5]:
X = None
Y = np.array([])
X, Y = read_files(X, Y, "/content/logloss_0", 0)
X, Y = read_files(X, Y, "/content/logloss_1", 1)

data = MyClass(X, Y)
train_data, test_data = torch.utils.data.random_split(data, [int(0.8*len(data)), int(0.2*len(data))])

In [6]:
data.__len__()

65

In [7]:
train_data.__len__()

52

In [8]:
loader_train = DataLoader(dataset=train_data, batch_size=5, shuffle=True)
loader_test = DataLoader(dataset=test_data, shuffle=True)

In [9]:
model = nn.Sequential(
    nn.Linear(65536, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(1024, 2),
    nn.Softmax()
)

In [11]:
pip install torch-summary

In [12]:

from torchsummary import summary
summary(model)

TypeError: ignored

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [14]:
model.cuda()

Sequential(
  (0): Linear(in_features=65536, out_features=1024, bias=True)
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.2, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.2, inplace=False)
  (7): ReLU()
  (8): Linear(in_features=1024, out_features=1024, bias=True)
  (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.2, inplace=False)
  (11): ReLU()
  (12): Linear(in_features=1024, out_features=2, bias=True)
  (13): Softmax(dim=None)
)

In [15]:
epochs = 20
model.train()
for i in range(epochs):
    for j, (x, y) in enumerate(loader_train):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x.float())
        loss = criterion(y_pred, y.long())
        print(f"Epoch {i}\t iter {j}\t loss {loss}")
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0	 iter 0	 loss 0.7851999998092651
Epoch 0	 iter 1	 loss 0.7686403393745422
Epoch 0	 iter 2	 loss 0.6015312075614929
Epoch 0	 iter 3	 loss 0.6214414834976196
Epoch 0	 iter 4	 loss 0.42758139967918396
Epoch 0	 iter 5	 loss 0.38301652669906616
Epoch 0	 iter 6	 loss 0.7105097770690918
Epoch 0	 iter 7	 loss 0.867392361164093
Epoch 0	 iter 8	 loss 0.6012037992477417
Epoch 0	 iter 9	 loss 0.8334954380989075
Epoch 0	 iter 10	 loss 0.8089786767959595
Epoch 1	 iter 0	 loss 0.31854376196861267
Epoch 1	 iter 1	 loss 0.6660093069076538
Epoch 1	 iter 2	 loss 0.3622705936431885
Epoch 1	 iter 3	 loss 0.7964896559715271
Epoch 1	 iter 4	 loss 0.7020136117935181
Epoch 1	 iter 5	 loss 0.4978969991207123
Epoch 1	 iter 6	 loss 0.44476231932640076
Epoch 1	 iter 7	 loss 0.5152632594108582
Epoch 1	 iter 8	 loss 0.5052732229232788
Epoch 1	 iter 9	 loss 0.3243638873100281
Epoch 1	 iter 10	 loss 1.2230174541473389
Epoch 2	 iter 0	 loss 0.33779382705688477
Epoch 2	 iter 1	 loss 0.7509219646453857
Epoch 2	 i

In [17]:
model.eval()
preds = []
true = []

for (x,y) in loader_test:
    x = x.cuda()
    y = y.cuda()
    preds.append(int(torch.max(model(x.float()).data, 1)[1]))
    true.append(int(y))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [18]:
print(true)
print(preds)

accuracy_score(true, preds)

[1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1]


0.9230769230769231

In [147]:
!nvidia-smi

Sat Jan 16 16:01:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    61W / 149W |  11366MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------